In [2]:
import sys 
sys.path.append('../../')
from src.import_packages import *

from bs4 import BeautifulSoup
import requests

Importing the following packages: pandas, numpy, matplotlib, seaborn, plotly, warnings


### Síterepek listája

In [3]:
url = 'https://sielok.hu/sitereplista/orszag/ausztria/listadizajn/rendabcnov/'
page = requests.get(url)
soup = BeautifulSoup(page.text)

In [59]:
siterepek = {}

for i in soup.find_all('tr'):
    _td = i.find_all('td')
    if (len(_td) != 0) and ('siterep' in _td[2].find('a')['href']):
        siterepek[_td[2].text] = 'https://sielok.hu' + _td[2].find('a')['href']

In [60]:
len(siterepek)

249

### Síterep infók

In [234]:
def siterep_infok(nev):

    i_ = siterepek[nev]
    page = requests.get(i_)
    soup = BeautifulSoup(page.text) 
    
    magassag = soup.find_all('div', class_ = 'tdc2 tdc')[0].text.replace('m', '').strip()
    hegyteto = int(magassag.split('-')[1].strip())
    parkolo = int(magassag.split('-')[0].strip())

    sipalyak_hossza = soup.find_all('div', class_ = 'tdc2 tdc')[1].text
    sipalyak_hossza = float(sipalyak_hossza.replace('km', '').strip().replace(',', '.'))

    szintkulonbseg = soup.find_all('div', class_ = 'tdc4 tdc')[0].text
    szintkulonbseg = float(szintkulonbseg.replace('m', '').strip().replace(',', '.'))

    liftek_szama = soup.find_all('div', class_ = 'tdc4 tdc')[1].text

    liftek = {}
    for i in soup.find_all('div', class_ = 'liftIkonPart')[0]:
        liftek[i.get('title')] = int(i.text)

    szinek = {0 : 'Kék', 1 : 'Piros', 2 : 'Fekete'}
    palyak = {}
    for enum, i in enumerate(soup.find_all('div', class_ = 'palyaSzinDobWrap')[0]):
        i_text = i.text.strip().split(' ')[0].replace(',', '.')
        palyak[szinek[enum]] = float(i_text)

    if 'Síszezon időszaka' in soup.find_all('div', class_ = 'div32')[0].text:
        idoszak = soup.find_all('div', class_ = 'div32')[0].text.strip().replace('Síszezon időszaka: ', '').strip()
        szezonveg = idoszak.split('–')[1].strip()
        szezoneleje = idoszak.split('–')[0].strip()

    else:
        szezonveg = np.nan
        szezoneleje = np.nan

    tavolsag = soup.find_all('div', class_ = 'div32')[2].text.replace('\xa0', ' ').strip()
    tavolsag = float(tavolsag.replace('Budapest:', '').split('km')[0].strip())

    napijegy = soup.find_all('div', class_ = 'div32')[3].text.replace('\xa0', ' ').replace('\n', '').replace('EUR', '€').strip()
    napijegy = float(napijegy.split('€')[0].replace('Felnőtt napijegy: ', '').strip())

    regio = np.nan
    for enum, i in enumerate(soup.find_all('div', class_ = 'cont')[0]):
        if enum == 6:
            regio = i.text

    adatok = {'Síterep' : nev,
    'Link' : i_,
    'Régió' : regio,
    'Parkoló' : parkolo,
    'Hegytető' : hegyteto,
    'Sípályák hossza' : sipalyak_hossza,
    'Szintkülönbség' : szintkulonbseg,
    'Liftek száma' : liftek_szama,
    'Szezon eleje' : szezoneleje,
    'Szezon vége' : szezonveg,
    'Távolság Budapesttől' : tavolsag,
    'Felnőtt napijegy ára' : napijegy}

    adatok.update(liftek)
    adatok.update(palyak)

    return adatok

In [235]:
%%time

kihagyhato = []
collector = []

for i in siterepek.keys():
    try:
        collector.append(siterep_infok(i))
    except:
        kihagyhato.append(siterepek[i])

Wall time: 1min 26s


In [236]:
print(len(collector))
print(len(kihagyhato))

215
34


### Concat

In [270]:
data = pd.DataFrame(collector)
print(data.shape)

(215, 21)


In [271]:
data['Liftek száma'] = data['Liftek száma'].astype(int)

for i in data.filter(like = 'lift').columns:
    data[i].fillna(0, inplace = True)
    data[i] = data[i].astype(int)

data['Vonat'].fillna(0, inplace = True)
data['Vonat'] = data['Vonat'].astype(int)

data['Kék'].fillna(0, inplace = True)
data['Piros'].fillna(0, inplace = True)
data['Fekete'].fillna(0, inplace = True)

In [272]:
data.isnull().sum()

Síterep                  0
Link                     0
Régió                    0
Parkoló                  0
Hegytető                 0
Sípályák hossza          0
Szintkülönbség           0
Liftek száma             0
Szezon eleje            24
Szezon vége             24
Távolság Budapesttől     0
Felnőtt napijegy ára     0
Húzólift                 0
Kombilift                0
Kék                      0
Piros                    0
Fekete                   0
Ülőlift                  0
Kiskabinos lift          0
Nagykabinos lift         0
Vonat                    0
dtype: int64

In [273]:
data.loc[data['Kék'] > data['Sípályák hossza'], 'Kék'] = data.loc[data['Kék'] > data['Sípályák hossza'], 'Kék'] / 1000
data.loc[data['Piros'] > data['Sípályák hossza'], 'Piros'] = data.loc[data['Piros'] > data['Sípályák hossza'], 'Piros'] / 1000
data.loc[data['Fekete'] > data['Sípályák hossza'], 'Fekete'] = data.loc[data['Fekete'] > data['Sípályák hossza'], 'Fekete'] / 1000

In [274]:
data.describe(percentiles = [.01, .1, .9, .99]).T

,count,mean,std,min,1%,10%,50%,90%,99%,max
Parkoló,215.0,1034.302326,345.447897,421.0,491.10,672.8,980.0,1498.80,2120.32,2685.0
Hegytető,215.0,1792.730233,544.015953,780.0,828.18,1106.0,1810.0,2475.20,3244.68,3440.0
Sípályák hossza,215.0,31.289302,46.993363,1.0,1.00,3.0,15.0,73.00,265.66,302.0
Szintkülönbség,215.0,758.427907,468.307574,37.0,88.52,200.0,700.0,1398.40,1913.00,2261.0
Liftek száma,215.0,10.246512,13.146538,1.0,1.00,2.0,6.0,21.60,68.18,90.0
Távolság Budapesttől,215.0,605.558140,174.551535,293.0,298.28,377.8,589.0,841.00,888.72,896.0
Felnőtt napijegy ára,215.0,40.492558,12.184249,11.0,17.00,23.2,41.0,56.40,61.00,64.0
Húzólift,215.0,5.200000,4.996447,0.0,1.00,2.0,4.0,9.60,27.16,39.0
Kombilift,215.0,0.069767,0.273037,0.0,0.00,0.0,0.0,0.00,1.00,2.0
Kék,215.0,12.528837,19.835638,0.2,0.50,1.5,6.0,28.52,122.64,140.0


In [275]:
data.describe(include = 'O').T

,count,unique,top,freq
Síterep,215,215,Venet - Landeck - Zams - Fliess,1
Link,215,215,https://sielok.hu/siterep/konigsberg/,1
Régió,215,7,Tirol,76
Szezon eleje,191,29,2021.12.17.,35
Szezon vége,191,21,2022.04.18.,43


In [277]:
data['Régió'].value_counts()

Tirol             76
Stájerország      35
Salzburg          29
Vorarlberg        24
Karintia          22
Alsó-Ausztria     16
Felső-Ausztria    13
Name: Régió, dtype: int64

In [278]:
data.to_csv('data/data.csv', index = False)